<a href="https://colab.research.google.com/github/GeoLabUniLaSalle/Python/blob/main/Cryptographie_Algorithme_RSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorithme RSA**

Dans ce chapitre, nous allons découvrir un exemple de **chiffrement asymétrique**. L'idée est de ne pas utiliser la même clé pour chiffrer et déchiffrer un texte. Nous aurons une clé dite **publique** pour **encoder** le message, et une clé dite **privée** pour le **décoder**.

Cette méthode permet de distribuer des clés publiques et que seul le propriétaire de la clé privée soit capable de décoder les messages codés avec la clé publique.

Ici, nous allons découvrir le fonctionnement de l'**algorithme** **RSA** (*en référence à ses inventeurs Rivest Shamir et Adleman*).

Cet algorithme repose sur un principe mathématique simple mais ingénieux.

A partir de deux **nombres premiers** P et Q, on crée une clé publique N dont la valeur est égale à P x Q.

La clé privée s'obtient à partir de P et Q.

En théorie, il est possible de retrouver P et Q à partir de N en le factorisant.

En pratique, si P et Q sont suffisament grands, aucun ordinateur n'est capable de retrouver les valeurs, le nombre d'opérations nécessaires étant trop élevé pour une résolution par force brute (nombre d'opérations exponentiel en fonction de la taille des facteurs premiers).

L'algorithme RSA est utilisé dans beaucoup de protocoles de sécurité sur internet.

Voici le fonctionnement de l'algorithme RSA. Commençons par choisir des valeurs pour P et Q.

In [128]:
P = 13
Q = 19

Calculons N, le produit de P par Q.

In [129]:
N = P*Q
N

247

Choisissons E un nombre n'ayant pas de facteur commun avec R = (P-1)x(Q-1).

In [130]:
R = (P-1)*(Q-1)
R

216

In [132]:
import math

def decomposition(x):
  """
  Retourne la décomposition de x en facteurs premiers, sous la forme d'une liste
  """
  d = []
  for  i in range(2,int(math.sqrt(x)+1)):
    while x%i==0:
      x = int(x/i)
      d.append(i)
    if x>1:
      d.append(x)
  d=sorted(list(set(d)))
  return d

In [133]:
decomposition(R)

[2, 3, 27]

In [134]:
E = 11

La **clé publique** P est la paire (E,N)

In [135]:
P = (E,N)
P

(11, 247)

Choisissons maintenant D telque E x D % R = 1.

In [136]:
i=1
while i*E%R!=1:
  i+=1
D=i
print(D,'*',E,'%',R,'=',i*E%R)

59 * 11 % 216 = 1


Prenons le premier nombre qui remplisse cette condition : 59.

La **clé privée** Q est la paire (D,N).

In [137]:
Q = (D,N)
Q

(59, 247)

Afin d'encode ou de décoder un texte, nous allons avoir besoin de transformer les lettres du texte en nombres et inversement. Nous affecterons la valeur 1 à a, 2 à B, etc.

Importont le module nécessaire.

In [139]:
from string import ascii_lowercase
print(ascii_lowercase)

abcdefghijklmnopqrstuvwxyz


En cas d'erreur sur ce bloc d'instructions, lancez l'installation du module string, redémarrez le noyau, puis re-exécutez la dernière cellule de code.

In [ ]:
import sys  
!{sys.executable} -m pip install --user string

Nous allons créer des fonctions permettant de transformer une lettre un nombre M, et inversement.

In [140]:
lower = ascii_lowercase

In [152]:
def nombre(x):
  '''
  Retourne la lettre x sous la forme d'un nombre
  '''
  if x in lower:
    return lower.index(x)+1
  else:
    return 0

In [153]:
def lettre(x):
  '''
  Retourne le nombre x sous la forme d'une lettre
  '''
  if x>0:
    return lower[x-1]
  else:
    return ' '

In [154]:
print(nombre('b'))
print(nombre('z'))

2
26


In [155]:
print(lettre(2))
print(lettre(26))

b
z


Réalisons maintenant la fonction de chiffrement RSA dont le principe est, pour chaque lettre du texte à chiffrer M, de calculer C = M ^ E % N où (E,N) est la clé publique.

In [179]:
def codeRSA(x,cle):
  '''
  Retourne x chiffré avec l'algorithme RSA et la clé publique cle
  '''
  a = []
  for i in x:
    a.append(nombre(i)**cle[0]%cle[1])
  return a

In [180]:
C = codeRSA('bonjour',P)
print(C)

[72, 98, 222, 147, 98, 148, 151]


Pour décoder C, il suffit de calculer S = C ^ D % N où (D,N) est la clé privée.

In [184]:
def decodeRSA(x,cle):
  '''
  Retourne x déchiffré avec l'algorithme RSA et la clé privée cle
  '''
  a = ''
  for i in x:
    a+=lettre(i**cle[0]%cle[1])
  return a

In [185]:
S = decodeRSA(C,Q)
print(S)

bonjour


Cet algorithme de cryptographie repose sur la démonstration mathématique de l'égalité :


> (M ^ E % N) ^ D % N = M

